<h1><center>--Spam-Filtering--💌--🎭--</center></h1>

<center><img src= "https://images.unsplash.com/flagged/photo-1560854350-13c0b47a3180?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1142&q=80" alt ="Diabetes" style='width: 600px;'></center>

<h3>SPAM ?</h3>

<b>Simply, Message we don't like, like:</b>

* commercial solicitations.
* advertisements.
* chain letters.
* pyramid schemes.
* fraudulent offers.


<h3>OK! Why should I do a model for this ??</h3>

* classifing it by ourselves consume time, cost and effort.
* Human can't be accuracte every minute & spam don't stop

<h3>Ready ! :</h3>

* 📚 Libraries

* 📕 Load Data

* 🔍 EDA(1)

* 🧽 Data Cleannig

* 🔍 EDA(2)

* 🎬 Preprocessing

* 🏗️ Model building

* 📏 Perofrmance measurements

* 👌 Cross validation


# 📚 Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.metrics import plot_roc_curve, auc
from sklearn.model_selection import StratifiedKFold

# 📕 Load Data

In [ ]:
data=pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin-1', usecols = ['v1', 'v2'])

# 🔍 Exploratory Data Analysis (1)


In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
data.head()

💡 need to change col-names & extract nore useful information from data

In [ ]:
data.info()

💡 I will try to make it numeric data as possible as i can in cleaning step

In [ ]:
data.shape

### 🤔 is there any Duplicates ?

In [ ]:
data.duplicated().sum()

💡 must deleted in cleaning step

### 😨 What about NaN values?

In [ ]:
data.isnull().sum()

💡 no null values, Good!

### 😱 is it balanced data?

In [ ]:
classes = data.groupby('v1').count()
classes['v2']

In [ ]:
slice= list(round(data['v1'].value_counts(normalize = True), 1))
label= ["HAM", "SPAM"]
colors = ["blue", "red"]
explode = [0, 0.1]

plt.pie(slice, labels = label, colors = colors, explode = explode, shadow = True)

💡 Unfortunately not. it is <b>im</b>Balanced data.<br>

# 🧽 Data Cleaning | _Action begin_

#### Add relevant columns Names!

In [ ]:
data = data.rename({"v1": "class", "v2": "documents"}, axis = 1)
data.head(2)

#### transform classes into binary

In [ ]:
data['class'] = data['class'].map({'spam' : 1, 'ham' : 0}) #simple

✨ Done transformtion

#### Delete Duplicated data

In [ ]:
data.drop_duplicates(keep = 'first', inplace = True)
data.shape

✨ duplicated data solved

#### Apply UnderSampling

<img src= "https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/resampling.png"  alt ="Diabetes" style='width: 600px;' align="left" >

In [ ]:
counts_class_0, counts_class_1 = data['class'].value_counts() 

data_c0 = data[data['class'] == 0]
data_c1 = data[data['class'] == 1]

data_under_c0 = data_c0.sample(counts_class_1, random_state = 42)

data_under = pd.concat([data_under_c0, data_c1], axis = 0)

data_under.shape

In [ ]:
view = data_under['class'].value_counts()

slice= list(round(view, 1))
label= ["HAM", "SPAM"]
colors = ["blue", "red"]

print(view)
plt.pie(slice, labels = label, colors = colors, shadow = True, startangle  = 90)

✨ Now it became <b>Balanced Data</b>

----------------------------------------------------------------

### EDA: how is bigger spam messages or Ham messages ? 

----------------------------------------------------------------

#### getting numbers of characters

In [ ]:
data_under['Nom_chars'] = data_under['documents'].str.len()
data.columns

#### getting numbers of words

In [ ]:
data_under['Nom_words'] = data_under['documents'].str.split().apply(len)
data_under.columns

# 🔍 Exploratory Data Analysis (2)

In [ ]:
data_under.describe().apply(round)

In [ ]:
corel = data_under.corr(method = 'pearson').abs()
fig = plt.figure(figsize=(10,5))
sns.heatmap(corel, annot = True)

In [ ]:
round(data_under.groupby('class').agg([np.mean,np.min,np.max]))

In [ ]:
avg = data_under.groupby('class').mean()

avg.plot(kind='bar',figsize = (9, 8),title='ham VS spam', color = ['blue', 'red'])
plt.grid(True)

it shows that spam messages (= 1) has more numbers of characters and words. <br>
💡 <b>So Spam messages seems larger than Ham</b>

# 🎬 Preprocessing Data

#### Lower case

In [ ]:
data_under['pre_pro_doc'] = data_under['documents'].str.lower()

#### delete punctuations

In [ ]:
data_under['pre_pro_doc'] = data_under['pre_pro_doc'].str.replace(r'[^\w\s]+', '')

#### removing stop words using tockenization

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')

data_under['pre_pro_doc'] = data_under['pre_pro_doc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop) ]))

Perfect! we Got preprocessed data

# 🏗️ Model Building

#### Bag of Words & Splitting data

In [ ]:
count_vec  = CountVectorizer()

X_train, X_test, y_train, y_test = train_test_split(data_under['pre_pro_doc'], data_under["class"], test_size = 0.2)

training_data = count_vec.fit_transform(X_train)

testing_data = count_vec.transform(X_test)

#### Build & Train algorithm

In [ ]:
naive_bayes = MultinomialNB()

naive_bayes.fit(training_data, y_train)

predictions = naive_bayes.predict(testing_data)

# 📏 Performance Measurement

#### Checking ROC curve | Quality of Splitting

In [ ]:
#checking how good splitting it is:
from sklearn.metrics import roc_curve

y_pred_proba = naive_bayes.predict_proba(testing_data)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)

#create ROC curve

plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

💡 Roc curve show high performance on spiltting data

## Before see results, it might depress you, but... <b>Stay Tuned</b>😉

#### checking perfromance | Accuracy, Precison..etc

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#Report Printing
print(classification_report(y_test,predictions,target_names=['Ham','Spam']))

#Confusion Matrix
conv = confusion_matrix(y_test, predictions)
sns.heatmap(conv, annot = True, annot_kws = {"size" : 16})

💡 from Confusion Matrix it shows that algorithm perfect classify spam and Ham pretty well !

💡 Notice:<br> here we are more interested in  Precision becuause its focus on predictive positive that include<br> 
   false positive which is more importat than others for spam filtering

# 💯👌 Cross Validation

above we see that measuring perfromance change every time runing code.<br>
so i applied cross validation to get the best possible result.<br>

i also want to get classifiaction report for best mesured performance<br>
and  its confusion matrix so i created that function below to do that !

In [ ]:
tprs = [] #ROC
aucs = [] #AUC
class_repo_lst = [] #Classification reports
acc_repo = [] #accuracies
conv_lst = [] #Confusion matrices
accruacy_scores_lst = []

In [ ]:
from sklearn.metrics import accuracy_score

def cross_validate_Model(classifier ,choice):
    '''
    parameters : 
    
     1- classifier : Algorithm to train
     2- choice : choose what you see.
                 - 'r': Report classifation & Roc_Curve
                 - 'c': Confusion Matrix
    '''
    
    #----------Data------------------
    X = np.array(data_under["pre_pro_doc"])
    y = np.array(data_under['class'])
    X, y = X[y != 2], y[y != 2]
    #----------Variables-------------
    cv = StratifiedKFold(n_splits=20)
    mean_fpr = np.linspace(0, 1, 100)

    #--------------------------------
    fig, ax = plt.subplots()
    vec = CountVectorizer()

    for i, (train, test) in enumerate(cv.split(X, y)):
        
        
        training_data = vec.fit_transform(X[train])
        
        testing_data = vec.transform(X[test])

        classifier.fit( training_data, y[train])  #fitting algoirthm

        y_pred = classifier.predict(testing_data) #getting prediction
        
        accruacy_scores_lst.append(accuracy_score(y[test], y_pred)) #save all accuraies in list
        
        #=========================================================================
        repo = classification_report(y[test], y_pred, target_names=['Ham','Spam']) 
        class_repo_lst.append(repo) 
        acc_repo.append(float(repo[203:207])) #get accuracy on spam 
        indx = acc_repo.index(max(acc_repo))
        #=========================================================================== 
                                   
        if choice == 'c':
            conv_lst.append(confusion_matrix(y[test], y_pred))
        
        if choice == 'r':
            #------plot-every-fold----------------------------------------------------------------------------------
            viz = plot_roc_curve(classifier, testing_data, y[test], name="", alpha=0.3, lw=1, ax=ax)
            #-------------------------------------------------------------------------------------------------------

            interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
            interp_tpr[0] = 0.0

            tprs.append(interp_tpr)
            aucs.append(viz.roc_auc)

    if choice == 'r':
        #------plotting-AUC----------------------------------------------------------------
        ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)
        #----------------------------------------------------------------------------------

        mean_tpr = np.max(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)

        #------plotting-ROC------------------------------------------------------------------------------------------------------
        ax.plot(mean_fpr, mean_tpr, color='b', lw=2, alpha=.8)
        #-------------------------------------------------------------------------------------------------------------------------

        print(class_repo_lst[indx])


    
    if choice == 'c':
        conv = conv_lst[indx]
        sns.heatmap(conv, annot = True, annot_kws = {"size" : 16})
        plt.show()
    
    
    if choice == 'r':
        ax.set(title="Maximum ROC curve after cross validation")
        ax.get_legend().remove()
        plt.show()
      
    
    class_repo_lst.clear()
    acc_repo.clear()
    conv_lst.clear()
        
    return  "Max_Accuracy : " + str(round(np.array(accruacy_scores_lst).max(), 4)) + "%"                              

In [ ]:
cross_validate_Model( MultinomialNB() ,'r')

💡 Roc curve shows perfect spiltting data

In [ ]:
cross_validate_Model( MultinomialNB(), 'c')

💡 Confusion matrix shows high quality classifiaction

In [ ]:
from sklearn.model_selection import cross_val_score

BoW = count_vec.fit_transform(data_under["pre_pro_doc"])

cross_val_score(naive_bayes, BoW, data_under["class"], scoring="accuracy", cv = 20).max()

# Accuracy 98.46%

### so glad you got this far 😃 
## if you like please UPVOTE ❤️